In [1]:
import os
import numpy as np 
import pandas as pd 

In [2]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/stanford_politeness_stack_HFSE_random.csv

File './../../../dataset/stanford_politeness_stack_HFSE_random.csv' already there; not retrieving.



In [3]:
path = "./../../../dataset/stanford_politeness_stack_HFSE_random.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (1636, 388)


,meta.Binary,true_label,given_label,col_1,col_2,col_3,col_4,col_5,col_6,col_7,...,col_376,col_377,col_378,col_379,col_380,col_381,col_382,col_383,col_384,true_error
0,0,13.0,13,0.294276,-0.297200,-0.553753,-0.248645,0.126060,-0.337412,0.510197,...,-0.332158,-0.381023,-0.136220,0.376297,0.482508,-0.636344,0.713697,0.610461,0.029585,0
1,0,13.0,13,0.083789,-0.157076,-0.520031,-0.141461,-0.580630,-0.411622,-0.535961,...,0.098274,0.025979,-0.447405,0.179427,0.595395,0.218394,-0.089305,0.201709,-0.316124,0
2,-1,5.0,5,-0.424103,0.029967,-0.384091,-0.026923,0.218921,-0.220167,-0.151426,...,-0.641262,-0.333165,-0.299042,-0.000881,-0.001424,0.016369,0.289453,0.416605,0.102963,0
3,-1,13.0,13,-0.579372,0.213908,0.304594,-0.688134,-0.802742,-0.170828,-0.053886,...,-0.203845,-0.033090,0.415439,-0.006171,-0.200774,0.071192,0.390093,0.292111,-0.016878,0
4,0,13.0,17,0.018370,-0.296969,-0.138444,-0.141088,-0.594611,-0.362157,-0.149240,...,-0.180750,-0.065512,-0.156769,0.247639,0.273521,0.413701,-0.202294,0.432897,0.056803,0


In [4]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,meta.Binary,given_label,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,...,col_375,col_376,col_377,col_378,col_379,col_380,col_381,col_382,col_383,col_384
0,0,13,0.294276,-0.297200,-0.553753,-0.248645,0.12606,-0.337412,0.510197,0.164346,...,0.337913,-0.332158,-0.381023,-0.136220,0.376297,0.482508,-0.636344,0.713697,0.610461,0.029585
1,0,13,0.083789,-0.157076,-0.520031,-0.141461,-0.58063,-0.411622,-0.535961,0.294069,...,0.056149,0.098274,0.025979,-0.447405,0.179427,0.595395,0.218394,-0.089305,0.201709,-0.316124


In [5]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 
# model_fit = model.fit(X,y)
# predictions = model.predict(X)

##### saving the predictions ####################
# savepath = "./../predictions/model_predictions.npy"
# np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [6]:
from autogluon.tabular import TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [7]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1636
Train Data Columns: 385
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1256.38 MB
	Train Data (Original)  Memory Usage: 5.04 MB (0.4% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 384 | ['col_1', 'col_2', 'col_3', 'col_4', 'col_5', ...]
		('int',

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI,0.826921,0.102428,0.163763,0.058994,2.089212,0.163763,0.058994,2.089212,1,True,3
1,RandomForestMSE,0.798523,0.263221,0.042900,0.028525,16.944979,0.042900,0.028525,16.944979,1,True,2
2,WeightedEnsemble_L2,0.771562,0.279234,0.214713,0.091187,23.077481,0.001775,0.000483,0.030636,2,True,4
3,LightGBM,0.600643,0.260386,0.006275,0.003185,4.012654,0.006275,0.003185,4.012654,1,True,1


### Cross Validation fitting

In [8]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    1636
Train Data Columns: 385
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1283.97 MB
	Train Data (Original)  Memory Usage: 5.04 MB (0.4% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in 

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_BAG_L1,0.959867,0.147239,1.147601,0.447897,8.726441,1.147601,0.447897,8.726441,1,True,3
1,RandomForestMSE_BAG_L1,0.908321,0.328580,0.047097,0.108391,26.344223,0.047097,0.108391,26.344223,1,True,2
2,WeightedEnsemble_L2,0.872069,0.335213,1.251848,0.579224,61.548256,0.001440,0.000165,0.033952,2,True,4
3,LightGBM_BAG_L1,0.736316,0.315257,0.055710,0.022771,26.443640,0.055710,0.022771,26.443640,1,True,1


### predicting and saving arrays 

In [ ]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))